In [2]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np

# Étape 1: Chargement des données

# Exemple : chargement des données depuis un fichier csv

# print("Chargement des données...")
# df = pd.read_csv('votre_fichier.csv')
# print("Données chargées. Shape:", df.shape)

# Exemple : chargement des données depuis un fichier JSON

print("Chargement des données...")
df = pd.read_json('data/amenagements-cyclables.json')
print("Données chargées. Shape:", df.shape)

Chargement des données...
Données chargées. Shape: (116016, 19)


In [3]:
# Étape 2: Inspection initiale
print("Aperçu des données:")
df.head()

Aperçu des données:


,osm_id,nom,amenagement,cote_amenagement,sens,surface,arrondissement,bois,coronapiste,amenagement_temporaire,infrastructure_bidirection,voie_a_sens_unique,position_amenagement,vitesse_maximale_autorisee,date_export,source,st_length_shape,geo_shape,geo_point_2d
0,41469916,Boulevard de l'Hôpital,couloir bus ouvert aux vélos,gauche,sens contraire de la voie,Asphalte,13,non,non,non,oui,non,niveau chaussée,30,2023-01-01T02:00:00+01:00,OSMIUM,22.698282,"{'type': 'Feature', 'geometry': {'coordinates'...","{'lon': 2.36181781113954, 'lat': 48.8398969824..."
1,59656043,None,piste cyclable,droit,None,None,12,non,non,non,oui,oui,indéfini,None,2023-01-01T02:00:00+01:00,OSMIUM,7.047969,"{'type': 'Feature', 'geometry': {'coordinates'...","{'lon': 2.376365099878524, 'lat': 48.838981249..."
2,85735923,Rue de Bercy,couloir bus ouvert aux vélos,droit,None,Asphalte,12,non,non,non,non,non,niveau chaussée,30,2023-01-01T02:00:00+01:00,OSMIUM,24.404972,"{'type': 'Feature', 'geometry': {'coordinates'...","{'lon': 2.37233855614266, 'lat': 48.8444140081..."
3,128522023,Rue Van Gogh,piste cyclable,gauche,sens contraire de la voie,Asphalte,12,non,non,non,oui,non,niveau trottoir,None,2023-01-01T02:00:00+01:00,OSMIUM,19.944198,"{'type': 'Feature', 'geometry': {'coordinates'...","{'lon': 2.370849008473838, 'lat': 48.843656978..."
4,150140702,None,piste cyclable,droit,None,None,12,non,non,non,oui,non,indéfini,None,2023-01-01T02:00:00+01:00,OSMIUM,62.403747,"{'type': 'Feature', 'geometry': {'coordinates'...","{'lon': 2.373003558759601, 'lat': 48.845530036..."


In [ ]:
print("\nInformations sur les colonnes:")
print(df.info())
print("\nStatistiques descriptives:")
print(df.describe())

In [ ]:
# Étape 3: Traitement des valeurs manquantes
print("\nValeurs manquantes avant traitement:")
print(df.isnull().sum())

# Remplacer les valeurs manquantes numériques par la médiane
numeric_columns = df.select_dtypes(include=[np.number]).columns
df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].median())

# Remplacer les valeurs manquantes catégorielles par le mode
categorical_columns = df.select_dtypes(include=['object']).columns
df[categorical_columns] = df[categorical_columns].fillna(df[categorical_columns].mode().iloc[0])

print("Valeurs manquantes après traitement:")
print(df.isnull().sum())

In [ ]:
# Étape 4: Vérification et conversion des types de données
# Exemple : conversion d'une colonne en type datetime
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'])

# Exemple : conversion d'une colonne en type catégoriel
if 'category' in df.columns:
    df['category'] = df['category'].astype('category')

print("\nTypes de données après conversion:")
print(df.dtypes)

In [ ]:
# Étape 5: Détection et traitement des valeurs aberrantes
# Exemple pour les colonnes 'age' et 'salary'
for col in ['age', 'salary']:  # Ajustez ces colonnes selon votre dataset
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[col] = df[col].clip(lower_bound, upper_bound)
print("\nValeurs aberrantes traitées pour les colonnes: age, salary")

In [ ]:
# Étape 6: Création de nouvelles caractéristiques (exemple)
if 'height' in df.columns and 'weight' in df.columns:
    df['bmi'] = df['weight'] / (df['height'] / 100) ** 2
    print("\nNouvelle caractéristique 'bmi' créée")

In [ ]:
# Étape 7: Sélection des caractéristiques
# Exemple: suppression de la colonne 'id' si elle existe
if 'id' in df.columns:
    df = df.drop(columns=['id'])
    print("\nColonne 'id' supprimée")

print("\nNettoyage des données terminé!")
print("Dimensions finales du dataframe:", df.shape)

In [ ]:
# Sauvegarde du dataframe nettoyé (optionnel)
df.to_csv('donnees_nettoyees.csv', index=False)
print("\nDonnées nettoyées sauvegardées dans 'donnees_nettoyees.csv'")